In [1]:
"""
Đọc & tiền xử lý dữ liệu cho tối ưu layout.

Chức năng chính:
- Đọc PPTX và trích xuất vị trí, kích thước hình chữ nhật (mm) + Category
- Nhận diện Entrance/Cashier (giữ nguyên vị trí), chỉ lấy các slot bày hàng để tối ưu
- Đánh dấu slot lạnh/không lạnh dựa vào danh sách refrigerated_categories
- Chuẩn hóa rules (cặp liên quan) + tính support (từ transactions × sku)
- Chuẩn bị dữ liệu cho SA (tọa độ slot bày hàng, danh sách category, slot lạnh, …)

Nguyên tắc:
- KHÔNG sắp xếp Entrance/Cashier.
- Các nhóm hàng “lạnh” chỉ được đặt vào slot lạnh; nhóm thường chỉ vào slot thường.
"""

import ast
import random
import unicodedata
import numpy as np
import pandas as pd
from collections import defaultdict
from pptx import Presentation
from pathlib import Path
from typing import Dict, Tuple, List, Optional

from src.config import (
    EXTERNAL_DATA_DIR,
    RAW_DATA_DIR,
    PROCESSED_DATA_DIR,
)


# -----------------
# Helpers
# -----------------
def normalize(s: str) -> str:
    """Chuẩn hóa chuỗi: lower, bỏ dấu, bỏ khoảng trắng đầu/cuối."""
    if s is None:
        return ""
    s = s.strip().lower()
    return "".join(
        ch for ch in unicodedata.normalize("NFD", s) if unicodedata.category(ch) != "Mn"
    )


def load_shapes_from_ppt_mm(ppt_file: Path) -> Tuple[pd.DataFrame, Presentation]:
    """Đọc tất cả shapes có text từ PPTX, trả về dataframe [x,y,width,height] theo mm."""
    EMU_PER_MM = 914400 / 25.4
    prs = Presentation(ppt_file)
    rows = []
    for s_idx, slide in enumerate(prs.slides):
        for sh_idx, shape in enumerate(slide.shapes):
            if not shape.has_text_frame:
                continue
            txt = (shape.text_frame.text or "").strip()
            if not txt:
                continue
            rows.append(
                {
                    "slide_idx": s_idx,
                    "shape_idx": sh_idx,
                    "shape_obj": shape,
                    "Category": txt,
                    "x": shape.left / EMU_PER_MM,
                    "y": shape.top / EMU_PER_MM,
                    "width": shape.width / EMU_PER_MM,
                    "height": shape.height / EMU_PER_MM,
                }
            )
    if not rows:
        raise RuntimeError("No shapes with text found in PPTX.")
    df = pd.DataFrame(rows)
    return df, prs


def rect_center(row: pd.Series) -> Tuple[float, float]:
    """Tâm hình chữ nhật (mm)."""
    return (row["x"] + row["width"] / 2.0, row["y"] + row["height"] / 2.0)


def parse_itemset(cell: str) -> Optional[str]:
    """
    Parse cột antecedent/consequent nếu đôi khi được ghi dạng ['A'].
    Trả về 1 string nếu là singleton, ngược lại None.
    """
    if pd.isna(cell):
        return None
    s = str(cell).strip()
    if not (s.startswith("[") and s.endswith("]")):
        return s if s else None
    try:
        val = ast.literal_eval(s)
        if isinstance(val, (list, tuple)) and len(val) == 1:
            return str(val[0])
        return None
    except Exception:
        inner = s.strip("[]").strip()
        parts = [p.strip(" '\"") for p in inner.split(",") if p.strip(" '\"")]
        return parts[0] if len(parts) == 1 else None


# -----------------
# Main loader
# -----------------
def load_input_data(
    layout_pptx: str = "layout.pptx",
    assoc_rules_csv: str = "association_rules.csv",
    transactions_csv: str = "transactions.csv",
    sku_csv: str = "sku.csv",
    refrigerated_categories: Optional[List[str]] = None,
    random_seed: int = 42,
) -> Dict:
    """
    Load + preprocess (PPTX geometry + CSV flags + rules + support).

    Quan trọng:
    - Nhận diện Entrance/Cashier bằng từ khóa: 'entry', 'entrance' và 'cashier' (bỏ dấu, không phân biệt hoa thường).
    - Chỉ các slot bày hàng (không phải Entrance/Cashier) được đưa vào tối ưu.
    - slot_is_cold: True nếu slot hiện tại chứa category thuộc danh sách 'refrigerated_categories' (tức vị trí đó là khu lạnh).
    - current_assign: ánh xạ Category->slot_idx ban đầu (để làm xuất phát cho SA).
    """
    random.seed(random_seed)
    np.random.seed(random_seed)
    refrigerated_categories = refrigerated_categories or []
    refrigerated_norm = {normalize(r) for r in refrigerated_categories}

    # 1) PPTX (đúng thư mục layout)
    df_ppt, prs = load_shapes_from_ppt_mm(EXTERNAL_DATA_DIR / layout_pptx)
    df_ppt["cx"], df_ppt["cy"] = zip(*df_ppt.apply(rect_center, axis=1))

    # 2) Flags (Entrance/Cashier/Cold)
    def is_entrance_kw(c: str) -> bool:
        n = normalize(c)
        return n in {"entry", "entrance"}

    def is_cashier_kw(c: str) -> bool:
        n = normalize(c)
        return n == "cashier"

    def is_cold_kw(c: str) -> bool:
        return normalize(c) in refrigerated_norm

    df = df_ppt.copy()
    df["is_entrance"] = df["Category"].apply(is_entrance_kw)
    df["is_cashier"] = df["Category"].apply(is_cashier_kw)
    df["is_refrigerated"] = df["Category"].apply(is_cold_kw)

    # 3) Bắt buộc có đúng 1 Entrance & 1 Cashier
    if df["is_entrance"].sum() != 1:
        raise RuntimeError(
            f"Expected exactly one Entrance (entry/entrance), found {df['is_entrance'].sum()}. "
            f"Check PPTX shape names!"
        )
    if df["is_cashier"].sum() != 1:
        raise RuntimeError(
            f"Expected exactly one Cashier, found {df['is_cashier'].sum()}. "
            f"Check PPTX shape names!"
        )

    entr_xy = (
        df.loc[df["is_entrance"], "cx"].iloc[0],
        df.loc[df["is_entrance"], "cy"].iloc[0],
    )
    cash_xy = (
        df.loc[df["is_cashier"], "cx"].iloc[0],
        df.loc[df["is_cashier"], "cy"].iloc[0],
    )

    # 4) Slots bày hàng = tất cả trừ Entrance/Cashier
    slots_mask = ~(df["is_entrance"] | df["is_cashier"])
    slots = df.loc[slots_mask].copy()
    slots["orig_idx"] = slots.index  # <— thêm dòng này: lưu index gốc trong df
    slots = slots.reset_index(drop=True)

    slots["slot_idx"] = np.arange(len(slots))
    layout_cats = slots["Category"].astype(str).tolist()
    # slot_is_cold: slot hiện tại đang là khu lạnh hay không, dựa theo category đang nằm ở đó
    slot_is_cold = slots["is_refrigerated"].astype(bool).tolist()

    # 5) Association rules (1→1)
    rules = pd.read_csv(PROCESSED_DATA_DIR / assoc_rules_csv)
    rules["_a"] = rules["antecedent"].apply(parse_itemset)
    rules["_b"] = rules["consequent"].apply(parse_itemset)
    pairs = rules.dropna(subset=["_a", "_b"]).copy()
    pairs["weight"] = pairs["lift"] if "lift" in pairs.columns else pairs["confidence"]
    pairs = pairs[["_a", "_b", "weight"]].rename(columns={"_a": "cat_a", "_b": "cat_b"})
    pairs = pairs[pairs["cat_a"].isin(layout_cats) & pairs["cat_b"].isin(layout_cats)]
    pairs = pairs[pairs["cat_a"] != pairs["cat_b"]]
    pairs = pairs.groupby(["cat_a", "cat_b"], as_index=False)["weight"].mean()
    pairs_list = [
        (a, b, w) for a, b, w in zip(pairs["cat_a"], pairs["cat_b"], pairs["weight"])
    ]

    # 6) Transactions × SKU -> baskets + support
    baskets = []
    cat_support = {}
    try:
        tx = pd.read_csv(RAW_DATA_DIR / transactions_csv, usecols=["Sku", "MergedId"])
        sku = pd.read_csv(RAW_DATA_DIR / sku_csv, usecols=["Sku", "SDeptName"])
        tx["Sku"] = tx["Sku"].astype(str)
        sku["Sku"] = sku["Sku"].astype(str)
        sku["SDeptName"] = sku["SDeptName"].astype(str)
        tx = tx.merge(sku, on="Sku", how="left").dropna(
            subset=["SDeptName", "MergedId"]
        )
        baskets = (
            tx.groupby("MergedId")["SDeptName"]
            .apply(lambda s: set(map(str, s.unique())))
            .tolist()
        )
        from collections import defaultdict as _dd

        cat_counts = _dd(int)
        for b in baskets:
            for c in b:
                cat_counts[c] += 1
        n_baskets = max(1, len(baskets))
        cat_support = {c: cat_counts.get(c, 0) / n_baskets for c in layout_cats}
    except FileNotFoundError:
        baskets = []
        cat_support = {c: 0.0 for c in layout_cats}

    # 7) Dữ liệu cho tối ưu
    coords = np.array(list(zip(slots["cx"].values, slots["cy"].values)))
    current_assign = {str(r["Category"]): i for i, r in slots.iterrows()}

    # Tập category lạnh ở THỜI ĐIỂM BAN ĐẦU (để ràng buộc giữ nhóm lạnh)
    cold_cats = set(
        str(r["Category"]) for _, r in slots.iterrows() if r["is_refrigerated"]
    )

    if len(cold_cats) > int(sum(slot_is_cold)):
        raise RuntimeError(
            f"Number of refrigerated categories ({len(cold_cats)}) exceeds refrigerated slots ({int(sum(slot_is_cold))})."
        )

    return {
        "df": df,
        "prs": prs,
        "entr_xy": entr_xy,
        "cash_xy": cash_xy,
        "slots": slots,
        "layout_cats": layout_cats,
        "slot_is_cold": slot_is_cold,
        "pairs_list": pairs_list,
        "cat_support": cat_support,
        "coords": coords,
        "current_assign": current_assign,
        "cold_cats": cold_cats,
        "baskets": baskets,
    }


refrigerated_categories = [
    "Thit dong lanh",
    "Tau hu cac loai",
    "Kem cac loai",
    "Tru mat khac (FLAN)",
    "San pham che bien d.lanh",
    "Com, xoi dong lanh",
    "Rau,cu,trai cay dong lanh",
    "Cha gio",
    "San pham ch.bien dong goi",
    "Hai san dong lanh",
]

input_data = load_input_data(
    layout_pptx="layout.pptx",
    assoc_rules_csv="association_rules.csv",
    transactions_csv="transactions.csv",
    sku_csv="sku.csv",
    refrigerated_categories=refrigerated_categories,
    random_seed=42,
)

c:\Users\lthnhung\AppData\Local\Programs\Python\Python311\Lib\site-packages\tsfresh\__init__.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
2025-08-11 15:26:27.439 | INFO     | src.config:<module>:15 - PROJ_ROOT path is: D:\DataLocal\lthnhung\My Documents\GitHub\Retail-Layout-Optimization-with-ML-Metaheuristics


In [2]:
"""
Thuật toán tối ưu (Simulated Annealing) cho việc hoán đổi category giữa các slot bày hàng.

Nguyên tắc:
- KHÔNG bao gồm Entrance/Cashier trong tập tối ưu.
- Chỉ hoán đổi TRONG CÙNG NHÓM:
    + Nhóm lạnh (category ∈ cold_cats) chỉ hoán đổi giữa các slot lạnh (slot_is_cold=True).
    + Nhóm thường (category ∉ cold_cats) chỉ hoán đổi giữa các slot thường (slot_is_cold=False).
- Mục tiêu: kết hợp chi phí cặp (từ rules, khoảng cách Euclid) + khoảng cách tới Entrance (có trọng số theo support).

Việc “chỉ swap trong cùng nhóm” đảm bảo 2 nhóm được sắp xếp cùng nhau và không trộn lẫn.
"""

import math
import random
import numpy as np
from typing import Dict, List, Tuple, Set

# Trọng số mục tiêu + tham số SA
ALPHA_PAIR = 1.0
BETA_ENTRANCE = 1.0
GAMMA_SUPPORT = 0.7
SA_ITERS = 8000
SA_START_TEMP = 1.0
SA_END_TEMP = 0.01


def euclid(p: Tuple[float, float], q: Tuple[float, float]) -> float:
    return math.hypot(p[0] - q[0], p[1] - q[1])


def objective(
    assign_map: Dict[str, int],
    pairs_list: List[Tuple[str, str, float]],
    cat_support: Dict[str, float],
    coords: np.ndarray,
    entr_xy: Tuple[float, float],
    mean_dist: float,
) -> float:
    """
    Hàm mục tiêu = ALPHA_PAIR * (chi phí cặp) + BETA_ENTRANCE * (chi phí khoảng cách tới Entrance có trọng số support).
    Chuẩn hóa theo mean_dist để ổn định scale.
    """
    # 1) Chi phí cặp (càng gần càng tốt)
    pair_cost = 0.0
    for a, b, w in pairs_list:
        ia = assign_map.get(a)
        ib = assign_map.get(b)
        if ia is None or ib is None:
            continue
        pair_cost += w * euclid(coords[ia], coords[ib])
    pair_cost = pair_cost / max(1e-9, mean_dist)

    # 2) Chi phí Entrance (dựa support^GAMMA_SUPPORT)
    ent_cost = 0.0
    for c in assign_map:
        s = (cat_support.get(c, 0.0)) ** GAMMA_SUPPORT
        ent_cost += s * euclid(entr_xy, coords[assign_map[c]])
    ent_cost = ent_cost / max(1e-9, mean_dist * len(assign_map))

    return ALPHA_PAIR * pair_cost + BETA_ENTRANCE * ent_cost


def is_feasible(
    assign_map: Dict[str, int], slot_is_cold: List[bool], cold_cats: Set[str]
) -> bool:
    """
    Hợp lệ nếu:
    - slot lạnh chỉ chứa cat lạnh, slot thường chỉ chứa cat thường
    - không trùng slot
    """
    used = set()
    for c in assign_map:
        si = assign_map[c]
        if slot_is_cold[si] and c not in cold_cats:
            return False
        if (not slot_is_cold[si]) and c in cold_cats:
            return False
        if si in used:
            return False
        used.add(si)
    return True


def simulated_annealing(input_data: Dict) -> Dict[str, int]:
    """
    Tối ưu ánh xạ Category->slot_idx cho CÁC SLOT BÀY HÀNG (Entrance/Cashier không tham gia).
    Swap chỉ thực hiện TRONG cùng nhóm (lạnh↔lạnh, thường↔thường).
    """
    coords = input_data["coords"]
    pairs_list = input_data["pairs_list"]
    cat_support = input_data["cat_support"]
    entr_xy = input_data["entr_xy"]
    layout_cats = input_data["layout_cats"]
    slot_is_cold = input_data["slot_is_cold"]
    cold_cats = input_data["cold_cats"]
    current_assign = input_data["current_assign"]

    # Tính mean_dist để scale objective
    if len(coords) >= 2:
        rand_idx = np.random.choice(
            len(coords), size=(min(500, len(coords) * 2), 2), replace=True
        )
        mean_dist = np.mean([euclid(coords[i], coords[j]) for i, j in rand_idx])
    else:
        mean_dist = 1.0

    # Tách danh sách category theo nhóm (dựa trên cold_cats)
    cold_cat_list = [c for c in layout_cats if c in cold_cats]
    warm_cat_list = [c for c in layout_cats if c not in cold_cats]

    def temperature(t: int) -> float:
        frac = t / max(1, SA_ITERS - 1)
        return SA_START_TEMP * (SA_END_TEMP / SA_START_TEMP) ** frac

    assign = current_assign.copy()
    curr_cost = objective(assign, pairs_list, cat_support, coords, entr_xy, mean_dist)
    best_assign = assign.copy()
    best_cost = curr_cost

    for t in range(SA_ITERS):
        T = temperature(t)

        # Chọn nhóm để swap (50-50 giữa lạnh/thường, nếu nhóm rỗng thì chọn nhóm còn lại)
        if cold_cat_list and warm_cat_list:
            use_cold = random.random() < 0.5
        elif cold_cat_list:
            use_cold = True
        elif warm_cat_list:
            use_cold = False
        else:
            break

        group = cold_cat_list if use_cold else warm_cat_list
        if len(group) < 2:
            continue  # không đủ phần tử để swap

        a, b = random.sample(group, 2)
        ia, ib = assign[a], assign[b]

        # Vì đã chọn cùng nhóm, 2 slot này mặc định phải có cùng loại (lạnh/thường)
        # nhưng vẫn kiểm tra để an toàn (phòng dữ liệu cũ bất nhất)
        if slot_is_cold[ia] != slot_is_cold[ib]:
            continue

        # Thử swap
        assign[a], assign[b] = ib, ia
        if not is_feasible(assign, slot_is_cold, cold_cats):
            assign[a], assign[b] = ia, ib
            continue

        new_cost = objective(
            assign, pairs_list, cat_support, coords, entr_xy, mean_dist
        )
        delta = new_cost - curr_cost
        if (delta < 0) or (random.random() < math.exp(-delta / max(1e-9, T))):
            curr_cost = new_cost
            if new_cost < best_cost:
                best_cost = new_cost
                best_assign = assign.copy()
        else:
            # hoàn tác nếu không chấp nhận
            assign[a], assign[b] = ia, ib

    return best_assign


assign_map = simulated_annealing(input_data)

{'Banh': 11,
 'Banh phong, b.trang,snack': 28,
 'Banh(tuoi,mi)TS Cap103-10': 27,
 'Bia': 30,
 'Bot cac loai': 13,
 'Caphe': 9,
 'Cha gio': 63,
 'Com, gao, nep cac loai': 10,
 'Com, xoi dong lanh': 8,
 'Dau an': 15,
 'Dien nha': 16,
 'Do dung mot lan': 44,
 'Do dung nau an, uong': 24,
 'Do dung nha bep': 20,
 'Do dung s.hoat gia dinh': 50,
 'Dung cu phuc vu SK&SD': 52,
 'Duong': 49,
 'Gia vi CL(kho,uot, bngot)': 46,
 'Giay ve sinh': 3,
 'HPham giat,xa,xit thomvai': 25,
 'HPham va dung cu cao rau': 17,
 'Hai san dong lanh': 40,
 'Hang do hop': 2,
 'Hang muoi chua': 54,
 'Hang say kho': 61,
 'Hat, dau rang': 43,
 'Kem cac loai': 60,
 'Keo': 57,
 'Khac (rau cau,rong sun)': 23,
 'Khu mui co the': 32,
 'Khu mui khong khi': 29,
 'Long nao, HP diet ctrung': 39,
 'Mat ong': 14,
 'Muoi': 0,
 'Mut, bo hat': 48,
 'Ngu coc': 62,
 'Nuoc cham CL(NTuong,Nmam)': 12,
 'Nuoc giai khat khac': 42,
 'Nuoc ngot': 1,
 'Nuoc uong': 66,
 'Rau,cu,trai cay dong lanh': 59,
 'Ruou co nong do': 58,
 'SP ve sinh, tay

In [3]:
"""
Lưu kết quả + trực quan:
- Cập nhật Category mới CHO CÁC SLOT BÀY HÀNG theo mapping tối ưu (Entrance/Cashier giữ nguyên)
- Xuất CSV, PNG (preview dạng grid), và PPTX đã thay nhãn & màu cho slot bày hàng
- Benchmark chiều dài đường đi (Manhattan, greedy từ Entrance đến Cashier)
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from pptx import Presentation
from pptx.dml.color import RGBColor
from pptx.util import Pt
from pathlib import Path
from typing import Dict, List, Tuple
import random
import math

from src.config import OUTPUT_DATA_DIR

PADDING_RATIO = 0.06


def manhattan(p: Tuple[float, float], q: Tuple[float, float]) -> float:
    return abs(p[0] - q[0]) + abs(p[1] - q[1])


def greedy_path_length(
    points: List[Tuple[float, float]],
    start: Tuple[float, float],
    end: Tuple[float, float],
    dist_fn=manhattan,
) -> float:
    """Tính độ dài đường đi kiểu tham lam: từ start đi qua các điểm rồi đến end."""
    if not points:
        return dist_fn(start, end)
    unvisited = points[:]
    cur = start
    total = 0.0
    while unvisited:
        nxt_i = min(range(len(unvisited)), key=lambda i: dist_fn(cur, unvisited[i]))
        nxt = unvisited.pop(nxt_i)
        total += dist_fn(cur, nxt)
        cur = nxt
    total += dist_fn(cur, end)
    return total


def estimate_cell_size_from_layout(df: pd.DataFrame) -> float:
    """Ước lượng kích thước cell để rasterize layout thành lưới (phục vụ vẽ PNG)."""
    vals = []
    arr = df[["x", "y", "width", "height"]].values
    n = len(arr)
    for i in range(n):
        x0, y0, w0, h0 = arr[i]
        x0b, y0b = x0 + w0, y0 + h0
        for j in range(i + 1, n):
            x1, y1, w1, h1 = arr[j]
            x1b, y1b = x1 + w1, y1 + h1
            if not (y0b < y1 or y1b < y0):
                gx = max(0, x0 - x1b, x1 - x0b)
                if gx > 1:
                    vals.append(gx)
            if not (x0b < x1 or x1b < x0):
                gy = max(0, y0 - y1b, y1 - y0b)
                if gy > 1:
                    vals.append(gy)
    if vals:
        return max(1, int(min(vals)) // 2)
    min_dim = np.minimum(df["width"], df["height"])
    return (
        max(1, int(np.median(min_dim[min_dim > 0]) / 4)) if (min_dim > 0).any() else 5
    )


def rasterize_grid(df: pd.DataFrame, cell_size: float):
    """Chuyển layout (hình chữ nhật) thành lưới (grid id) để vẽ ảnh preview."""
    cats = list(df["Category"].astype(str).unique())
    name2id = {c: i + 1 for i, c in enumerate(cats)}
    id2name = {v: k for k, v in name2id.items()}

    x0, y0 = df["x"].min(), df["y"].min()
    x1, y1 = (df["x"] + df["width"]).max(), (df["y"] + df["height"]).max()
    pad_x, pad_y = int((x1 - x0) * PADDING_RATIO), int((y1 - y0) * PADDING_RATIO)
    min_x, min_y = x0 - pad_x, y0 - pad_y
    max_x, max_y = x1 + pad_x, y1 + pad_y

    W = int(math.ceil((max_x - min_x) / cell_size))
    H = int(math.ceil((max_y - min_y) / cell_size))
    if W * H > 1e7:  # an toàn bộ nhớ
        scale_factor = math.sqrt((W * H) / 1e7)
        cell_size *= scale_factor
        W = int(math.ceil((max_x - min_x) / cell_size))
        H = int(math.ceil((max_y - min_y) / cell_size))

    grid = np.zeros((H, W), dtype=np.int32)
    for _, r in df.iterrows():
        did = name2id[str(r["Category"])]
        gx0 = int(math.floor((r["x"] - min_x) / cell_size))
        gx1 = int(math.ceil((r["x"] + r["width"] - min_x) / cell_size))
        gy0 = int(math.floor((r["y"] - min_y) / cell_size))
        gy1 = int(math.ceil((r["y"] + r["height"] - min_y) / cell_size))
        grid[gy0:gy1, gx0:gx1] = did

    meta = {"min_x": min_x, "min_y": min_y, "cell_size": cell_size, "W": W, "H": H}
    return grid, name2id, id2name, meta


def visualize_pretty(df_layout: pd.DataFrame, out_png: Path) -> None:
    """Vẽ preview layout ra PNG (label tự động, giữ Entrance/Cashier nguyên vị trí)."""
    cell = estimate_cell_size_from_layout(df_layout)
    grid, name2id, id2name, meta = rasterize_grid(df_layout, cell)

    H, W = grid.shape
    unique_ids = np.unique(grid)
    max_id = int(unique_ids.max()) if len(unique_ids) > 0 else 0

    colors = ["#FFFFFF"] + [
        plt.cm.get_cmap("tab20", max(1, max_id))(i) for i in range(max_id)
    ]
    cmap = mcolors.ListedColormap(colors)
    bounds = list(range(0, max_id + 2))
    norm = mcolors.BoundaryNorm(bounds, cmap.N)

    fig_w = min(30, 18 * (W / max(1, H)))
    fig_h = min(30, 18)
    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    ax.imshow(grid, cmap=cmap, norm=norm, interpolation="none")

    for did in np.unique(grid[grid > 0]):
        ys, xs = np.where(grid == did)
        if len(xs) == 0:
            continue
        cx, cy = np.mean(xs), np.mean(ys)
        name = id2name.get(did, f"ID {did}")
        region_color = cmap(norm(did))
        lum = (
            0.299 * region_color[0] + 0.587 * region_color[1] + 0.114 * region_color[2]
        )
        txt_color = "white" if lum < 0.5 else "black"
        w = xs.max() - xs.min() + 1
        h = ys.max() - ys.min() + 1
        rot = 90 if h > w * 1.6 and len(name) > 5 else 0
        fontsize = max(6, min(11, int(np.sqrt(w * h) / max(1, len(name)) * 4)))
        ax.text(
            cx,
            cy,
            name,
            va="center",
            ha="center",
            color=txt_color,
            fontsize=fontsize,
            rotation=rot,
            weight="bold",
        )

    ax.set_title(
        "Layout mới (preview dạng grid từ PPTX + flags từ CSV)",
        fontsize=14,
        weight="bold",
    )
    ax.grid(False)
    ax.tick_params(
        axis="both",
        which="both",
        bottom=False,
        left=False,
        labelbottom=False,
        labelleft=False,
    )
    plt.savefig(out_png, dpi=220, bbox_inches="tight")
    plt.close(fig)


def apply_layout_to_ppt(
    df_new: pd.DataFrame, prs_in: Presentation, out_pptx: Path
) -> None:
    """
    Ghi lại nhãn Category cho CÁC SLOT BÀY HÀNG trên PPTX (không động vào Entrance/Cashier).
    Tô màu theo category để dễ phân biệt.
    """
    cats = list(
        df_new.loc[~(df_new["is_entrance"] | df_new["is_cashier"]), "Category"]
        .astype(str)
        .unique()
    )
    cmap = plt.cm.get_cmap("tab20", max(1, len(cats)))
    cat2rgb = {
        c: (int(r * 255), int(g * 255), int(b * 255))
        for i, c in enumerate(cats)
        for r, g, b, _ in [cmap(i)]
    }

    df_place = df_new[~(df_new["is_entrance"] | df_new["is_cashier"])].sort_values(
        ["slide_idx", "shape_idx"]
    )
    new_texts = df_place["Category"].tolist()
    shapes_sorted = df_place["shape_obj"].tolist()

    for sh, new_cat in zip(shapes_sorted, new_texts):
        sh.text_frame.clear()
        p = sh.text_frame.paragraphs[0]
        run = p.add_run()
        run.text = str(new_cat)
        run.font.size = Pt(10)
        run.font.bold = True
        if new_cat in cat2rgb:
            r, g, b = cat2rgb[new_cat]
            sh.fill.solid()
            sh.fill.fore_color.rgb = RGBColor(r, g, b)
            if sh.line:
                sh.line.color.rgb = RGBColor(40, 40, 40)
                sh.line.width = Pt(0.75)

    prs_in.save(out_pptx)


def save_and_visualize(
    input_data: Dict,
    assign_map: Dict[str, int],
    output_layout_csv: str = "layout_new.csv",
    output_preview_png: str = "layout_new.png",
    output_pptx: str = "layout_new.pptx",
) -> Dict:
    """
    Áp mapping tối ưu vào dataframe gốc (chỉ slot bày hàng), rồi xuất CSV/PNG/PPTX.
    Benchmark đường đi dựa trên baskets (nếu có).
    """
    df = input_data["df"].copy()
    slots = input_data["slots"]
    prs = input_data["prs"]
    entr_xy = input_data["entr_xy"]
    cash_xy = input_data["cash_xy"]
    baskets = input_data["baskets"]

    # cập nhật Category cho các slot bày hàng theo best_assign (giữ nguyên Entrance/Cashier)
    slotidx_to_cat = {assign_map[c]: c for c in input_data["layout_cats"]}

    for _, row in slots.iterrows():
        orig_idx = int(row["orig_idx"])  # <— dùng orig_idx để trỏ đúng hàng ở df
        slot_idx = int(row["slot_idx"])
        new_cat = slotidx_to_cat.get(slot_idx, row["Category"])
        df.at[orig_idx, "Category"] = new_cat  # <— chỉ cập nhật các slot bày hàng

    # (Tuỳ chọn) An toàn: khẳng định Entrance/Cashier không đổi
    entr_row = input_data["df"].loc[input_data["df"]["is_entrance"]].iloc[0]
    cash_row = input_data["df"].loc[input_data["df"]["is_cashier"]].iloc[0]
    assert (
        df.loc[entr_row.name, "Category"] == entr_row["Category"]
    ), "Entrance bị ghi đè!"
    assert (
        df.loc[cash_row.name, "Category"] == cash_row["Category"]
    ), "Cashier bị ghi đè!"

    # Xuất file
    (OUTPUT_DATA_DIR / output_layout_csv).parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(OUTPUT_DATA_DIR / output_layout_csv, index=False, encoding="utf-8-sig")
    visualize_pretty(df, OUTPUT_DATA_DIR / output_preview_png)
    apply_layout_to_ppt(df, prs, OUTPUT_DATA_DIR / output_pptx)

    # Benchmark (giữ nguyên Entrance/Cashier)
    orig_cat2xy = {
        str(r["Category"]): (r["cx"], r["cy"])
        for _, r in input_data["df"].iterrows()
        if not (r["is_entrance"] or r["is_cashier"])
    }
    new_cat2xy = {
        str(r["Category"]): (r["cx"], r["cy"])
        for _, r in df.iterrows()
        if not (r["is_entrance"] or r["is_cashier"])
    }

    def avg_path_length(cat2xy):
        sample_b = (
            baskets
            if len(baskets) <= 500
            else random.sample(baskets, 500) if baskets else []
        )
        lens = []
        for b in sample_b:
            pts = [cat2xy[c] for c in b if c in cat2xy]
            lens.append(greedy_path_length(pts, entr_xy, cash_xy, dist_fn=manhattan))
        return np.mean(lens) if lens else float("nan")

    base_L = avg_path_length(orig_cat2xy)
    new_L = avg_path_length(new_cat2xy)
    improve = (
        (base_L - new_L) / base_L * 100
        if (base_L and not math.isnan(base_L) and base_L > 0)
        else float("nan")
    )

    print(f"Base path length: {base_L:.2f}")
    print(f"New path length: {new_L:.2f}")
    print(f"Improvement: {improve:.2f}%")

    return {
        "df_new": df,
        "base_length": base_L,
        "new_length": new_L,
        "improvement": improve,
    }


save_and_visualize(
    input_data,
    assign_map,
    output_layout_csv="layout_new.csv",
    output_preview_png="layout_new.png",
    output_pptx="layout_new.pptx",
)

C:\Users\lthnhung\AppData\Local\Temp\ipykernel_69784\1021450154.py:120: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  plt.cm.get_cmap("tab20", max(1, max_id))(i) for i in range(max_id)


Base path length: 297.61
New path length: 289.81
Improvement: 2.62%


C:\Users\lthnhung\AppData\Local\Temp\ipykernel_69784\1021450154.py:188: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap("tab20", max(1, len(cats)))


{'df_new':     slide_idx  shape_idx                                          shape_obj  \
 0           0          0  <pptx.shapes.autoshape.Shape object at 0x00000...   
 1           0          1  <pptx.shapes.autoshape.Shape object at 0x00000...   
 2           0          2  <pptx.shapes.autoshape.Shape object at 0x00000...   
 3           0          3  <pptx.shapes.autoshape.Shape object at 0x00000...   
 4           0          4  <pptx.shapes.autoshape.Shape object at 0x00000...   
 ..        ...        ...                                                ...   
 64          0         64  <pptx.shapes.autoshape.Shape object at 0x00000...   
 65          0         65  <pptx.shapes.autoshape.Shape object at 0x00000...   
 66          0         66  <pptx.shapes.autoshape.Shape object at 0x00000...   
 67          0         67  <pptx.shapes.autoshape.Shape object at 0x00000...   
 68          0         68  <pptx.shapes.autoshape.Shape object at 0x00000...   
 
                      Catego